## Pre-processing Dati Covid-19

In [1]:
import pandas as pd
covid = pd.pandas.read_csv(r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
#manipolazione del dataframe per poterlo trasformare in un df "verticale"
dates = covid.columns[4:]
covid19 = covid.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates,
    var_name='Date', 
    value_name='Confirmed'
)
covid19['Date'] = pd.to_datetime(covid19['Date'])
C = pd.DataFrame(data=covid19)
c = C.set_index('Country/Region')
#filtraggio del dataframe tenendo solo i dati relativi degli stati di nostro interesse
filtring_df = c.loc[['Argentina','Australia','France','Turkey','Saudi Arabia','Italy','India','US','Canada','Japan','Germany','Spain','China','Russia','Brazil','Korea, South','United Kingdom']]
filtring =filtring_df.sort_index()
filtring= filtring.reset_index()
filtring = filtring.rename(columns={"Province/State":"Province"})
filtring['Province'].fillna(0, inplace=True)
#rimozione delle province di alcune nazioni aventi province al di fuori del mainland (vedasi UK o Francia)
df = filtring.drop(filtring[(filtring['Country/Region'] == 'United Kingdom') & (filtring['Province'] != 0)].index)
df_1=df.drop(df[(df['Country/Region'] == 'France') & (df['Province'] != 0)].index)
#altre nazioni (es Cina, Australia) erano suddivise in province, abbiamo sommato i risultati sotto il nome della nazione di provenienza
covid_19 =df_1.groupby(['Country/Region','Date'],as_index=False)['Confirmed'].sum()
covid_19.rename(columns={'Country/Region':'Country'}, inplace=True)
c19=covid_19.sort_values(by=['Date'],ascending=False)

#import dei dati su mongo
from pymongo import MongoClient
import pymongo
client=MongoClient()
db=client.dataman
covid = db.covid
records_ = c19.to_dict(orient = 'records')
result = db.covid.insert_many(records_ )